In [ ]:
%run mi_lsd_header_and_preprocessing_functions.py
## set path to data_lsd_trip
## HERE
drive = "/SETDRIVEPATH/data_lsd_trip/"

def train_rf_model(y, X, method, o_path, set_name, n_estimators = 100, max_depth = 30, criterion = "mse"):
    t = time.time()
    rf = RandomForestRegressor(n_estimators=n_estimators, 
                               max_depth=max_depth, n_jobs=15, 
                               random_state=None,
                               criterion=criterion)
    rf.fit(X, y)
    savepath = o_path+'rf_'+method+'_'+set_name+'_n'+str(n_estimators)+'_d'+str(max_depth)+'_c.'+criteron
    pickle.dump(rf, open(savepath, 'wb') )
    print("saved model to", savepath)
    print("elapsed time:", time.time()-t,"s")
    del(rf)

def train_nn_model(y, X, method, o_path, set_name, verbose = False, 
                   batch_size = 10000, epochs = 100, learning_rate=0.01, dropout_p=0.0):
    t = time.time()
    torch.manual_seed(1)
    cuda = torch.device("cuda:0")
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    torch.cuda.empty_cache()
    
    x, y = (Variable(torch.from_numpy(X[:,:].astype(np.float32)).cuda()), 
            Variable(torch.unsqueeze(torch.from_numpy(y[:].astype(np.float32)).cuda(), dim=1))
            )
    x = x.to(cuda)
    y = y.to(cuda)

    n_input = X.shape[1]

    net = torch.nn.Sequential(
        torch.nn.Linear(n_input, 2*n_input),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(2*n_input, 2*n_input),
        torch.nn.LeakyReLU(),
        torch.nn.Dropout(p=dropout_p),
        torch.nn.Linear(2*n_input, 2*n_input),
        torch.nn.LeakyReLU(),
        torch.nn.Dropout(p=dropout_p),
        torch.nn.Linear(2*n_input, 2*n_input),
        torch.nn.LeakyReLU(),
        torch.nn.Dropout(p=dropout_p),
        torch.nn.Linear(2*n_input, 1)
    )

    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    loss_func = torch.nn.L1Loss()
    torch_dataset = Data.TensorDataset(x, y)

    loader = Data.DataLoader(
        dataset=torch_dataset, 
        batch_size=batch_size, 
        shuffle=True)

    losses = []
    for epoch in range(epochs):
        for step, (batch_x, batch_y) in enumerate(loader): # for each training step
            lr = learning_rate * (0.9 ** (epoch/2))
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            
            b_x = Variable(batch_x)
            b_y = Variable(batch_y)
            prediction = net(b_x)     # predict based on x

            loss = loss_func(prediction, b_y)     # (1. nn output, 2. target)
            losses.append(loss)
            optimizer.zero_grad()   # clear gradients for next train
            loss.backward()         # backpropagation, compute gradients
            optimizer.step()        # apply gradients

        if epoch%10 == 0:
            print("Epoch", epoch,"/", epochs)
    net_cpu = net.cpu()
    del(net)
    savepath = o_path+'nn_'+method+'_'+set_name+'_b'+str(batch_size)+'_e'+str(epochs)+'_l'+str(learning_rate)+'_d'+str(dropout_p)
    torch.save(net_cpu, savepath)
    print("saved model to", savepath)
    print("elapsed time:", time.time()-t,"s")
    
    if verbose:
        fig, ax = plt.subplots(figsize=(6,6))
        plt.cla()
        ax.set_title('Regression Analysis', fontsize=18)
        ax.set_xlabel('gt', fontsize=14)
        ax.set_ylabel('est', fontsize=14)
        ax.set_xlim(-0.1, 1.1)
        ax.set_ylim(-0.1, 1.1)
        ax.scatter(y.data.cpu().numpy(), y.data.cpu().numpy(), color = "blue", alpha=0.1)

        prediction = net_cpu(x.data.cpu())
        ax.scatter(y.data.cpu().numpy(), prediction.data.cpu().numpy(), color='green', alpha=0.005)
        plt.savefig(o_path+'nn_'+method+'_'+set_name+'_end.png')
        plt.show()

        plt.figure(figsize=(6,3))
        plt.plot(np.log(losses))
        plt.savefig(o_path+'nn_'+method+'_'+set_name+'_losses.png')
        plt.show()

        foo = np.abs(y.data.cpu().numpy()-prediction.data.cpu().numpy())
        f = plt.hist(foo, bins=np.linspace(0,0.4,100), alpha=0.5)
        plt.show()
        print("mean abs err on train set:", np.mean(foo))
    del(net_cpu)

In [ ]:
## main results

set_name = "SET03_TRAIN"
i_path = drive+"data_in_silico/MI-LSD_"
o_path = drive+"trained_models/0-15-unsorted-augmented/"

method="lsd"
y, X, S = preprocess_np_for_lsd(i_path+set_name, start_wl_id=0, stop_wl_id=16)
train_rf_model(y, X, method, o_path, set_name, n_estimators = 100, max_depth = 30)
train_nn_model(y, X, method, o_path, set_name, epochs=100, verbose = True, learning_rate=0.01, dropout_p=0.0)
train_nn_model(y, X, method, o_path, set_name, epochs=100, verbose = True, learning_rate=0.01, dropout_p=0.2)

method="mi-lsd"
y, X = preprocess_np_for_milsd(i_path+set_name, start_wl_id=0, stop_wl_id=16, sort=False, data_augment=True)
train_rf_model(y, X, method, o_path, set_name, n_estimators = 100, max_depth = 30)
train_nn_model(y, X, method, o_path, set_name, epochs=100, verbose = True, learning_rate=0.01, dropout_p=0.0)
train_nn_model(y, X, method, o_path, set_name, epochs=100, verbose = True, learning_rate=0.01, dropout_p=0.2)

In [ ]:
## with only 13 walvelengths
set_name = "SET03_TRAIN"
i_path = drive+"data_in_silico/MI-LSD_"
o_path = drive+"trained_models/0-12-unsorted-augmented/"
method="lsd"
y, X, S = preprocess_np_for_lsd(i_path+set_name, start_wl_id=0, stop_wl_id=13)
train_rf_model(y, X, method, o_path, set_name, n_estimators = 100, max_depth = 30)
train_nn_model(y, X, method, o_path, set_name, epochs=100, verbose = True, learning_rate=0.01, dropout_p=0.0)

method="mi-lsd"
y, X = preprocess_np_for_milsd(i_path+set_name, start_wl_id=0, stop_wl_id=13, sort=False, data_augment=True)
train_rf_model(y, X, method, o_path, set_name, n_estimators = 100, max_depth = 30)
train_nn_model(y, X, method, o_path, set_name, epochs=100, verbose = True, learning_rate=0.01, dropout_p=0.0)